In [14]:
from api_executer import ApiExecuter
from api_requester import ApiRequester
from api_parser import ApiParser
from functions import get_current_location

In [15]:
location = get_current_location()
if location:
    latitude, longitude = location
    print(f'Current Latitude: {latitude}, Longitude: {longitude}')
else:
    print('Failed to determine current location.')

Current Latitude: -34.9287, Longitude: 138.5986


In [16]:
from route_csv2df import RouteDF
routeDF = RouteDF()

from api_route import ApiRoute
current_position = {'longitude': 130.844161,
                    'latitude': -12.456694}
final_position = {'longitude': 130.890346,
                  'latitude': -12.432393}
delta_spacing = 2000.0 # in meters
number_sites = 10

route = ApiRoute(routeDF.get_route_data)

route_df = route.cut_route_data(current_position, number_sites=number_sites)
# route_df = route.cut_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing, print_is_requested=True)
# route_df = route.cut_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites)

Nearest point index: 8
    longitude   latitude  altitudeSmoothed  inclinationSmoothed   cumDistance  \
0  130.844161 -12.456694         16.350559            -0.024998  4.070956e+02   
1  132.432250 -14.530751        177.501908            -0.010896  3.371206e+05   
2  133.443126 -17.041297        244.299944            -0.000802  6.738341e+05   
3  134.202227 -19.844744        333.407446             0.001097  1.010548e+06   
4  133.331310 -22.515267        638.287834             0.001966  1.347261e+06   
5  133.198145 -25.097933        460.958486             0.008386  1.683975e+06   
6  134.024609 -27.828917        300.423269            -0.002393  2.020688e+06   
7  135.350570 -30.414686        170.808656            -0.000158  2.357402e+06   
8  137.588876 -32.317265         44.598942            -0.000095  2.694115e+06   
9  138.608989 -34.935232         56.847434            -0.016969  3.030829e+06   

   maxSpeed       theta  cumDistanceCut  
0        50   44.136411    0.000000e+00  
1

In [17]:
api_parser = ApiParser()
api_requester = ApiRequester(parser=api_parser)
api_executer = ApiExecuter(requester=api_requester)

Response status from siteinfo: OK.
Current sites' info has been retrieved. 
         name  longitude  latitude  altitude      UTC_offset
site_id                                                    
586195     0    130.844  -12.4567        32 0 days 09:30:00
586196     1    132.432  -14.5308       176 0 days 09:30:00
586197     2    133.443  -17.0413       248 0 days 09:30:00
586198     3    134.202  -19.8447       332 0 days 09:30:00
586199     4    133.331  -22.5153       640 0 days 09:30:00
586200     5    133.198  -25.0979       458 0 days 09:30:00
586201     6    134.025  -27.8289       302 0 days 09:30:00
586202     7    135.351  -30.4147       169 0 days 09:30:00
586203     8    137.589  -32.3173        46 0 days 09:30:00
586204     9    138.609  -34.9352        52 0 days 09:30:00


In [18]:
# api_requester.get_site_info(print_is_requested=True)

# api_executer.get_all_site_id


In [19]:
# api_executer.add_sites(route_df, print_is_requested=True)
sites_df = api_requester.forecast_sites

In [20]:
# api_requester.get_site_delete(585394, print_is_requested=True)
# api_executer.delete_all_sites()

In [21]:
# import pandas as pd
# import numpy as np
# import random
# import string

# sites_list = api_executer.get_all_site_id(print_is_requested=False)
# s_len = len(sites_list)

# # Create the DataFrame
# prob = 1.0
# data = {
#     # 'name': ['e'+str(random.randint(1,10)) for _ in range(s_len)],
#     'latitude': [random.uniform(-90.0, 90.0) if random.random() < prob else np.nan for _ in range(s_len)],
#     'longitude': [random.uniform(-180.0, 180.0) if random.random() < prob else np.nan for _ in range(s_len)]
# }

# edit_sites = pd.DataFrame(data, index=sites_list)
# # print(edit_sites)

# api_executer.edit_sites(edit_sites, print_is_requested=True)


In [22]:
# raw_forecast_df = api_requester.get_solar_forecast()
raw_forecast_df = api_requester.get_solar_forecast_cloudmove()
print(raw_forecast_df)

Response status from getforecast_cloudmove: OK.
Solar forecast CloudMove have been retrieved.
                                           gk      tt          gh         dh  \
site_id time                                                                   
586195  2023-09-26 09:15:00+09:30  598.000000  28.100  598.000000  66.366363   
        2023-09-26 09:30:00+09:30  659.000000  28.400  659.000000  71.811203   
        2023-09-26 09:45:00+09:30  721.000000  28.750  721.000000  75.355095   
        2023-09-26 10:00:00+09:30  775.500000  29.100  775.500000  81.054956   
        2023-09-26 10:15:00+09:30  826.000000  29.450  826.000000  87.353706   
...                                       ...     ...         ...        ...   
586204  2023-09-26 14:00:00+09:30  788.970347  21.050  788.970347  79.091126   
        2023-09-26 14:15:00+09:30  757.509185  21.125  757.509185  75.432021   
        2023-09-26 14:30:00+09:30  722.332496  21.200  722.332496  71.710568   
        2023-09-26 14:45:0

In [23]:
from preprocessor import Preprocessor
preprocessor = Preprocessor()
hours_in_advance = 4
forecast_df, preprocessed_df = preprocessor.preprocess(route_df, sites_df, raw_forecast_df, hours_in_advance)
preprocessor.save_data()
print(preprocessed_df)

Data saved to G:/Drive condivisi/AlphaCentauri/SolarCar_22 23/6. Strategy & Simulation/ss_online_data\20230926_091239_CM.
                                        temperature  globalIrradiance
cumDistance  time                                                    
4.070956e+02 2023-09-26 09:15:00+09:30        28.10           598.000
             2023-09-26 09:30:00+09:30        28.40           659.000
             2023-09-26 09:45:00+09:30        28.75           721.000
             2023-09-26 10:00:00+09:30        32.10           775.500
             2023-09-26 10:15:00+09:30        32.45           826.000
...                                             ...               ...
3.030829e+06 2023-09-26 12:15:00+09:30        23.60           895.197
             2023-09-26 12:30:00+09:30        23.70           892.767
             2023-09-26 12:45:00+09:30        23.75           886.011
             2023-09-26 13:00:00+09:30        23.80           874.958
             2023-09-26 13:15:00+09:30